# Data merging

### Import required packages

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings("ignore")

### Import required dataset into pandas dataframe

In [ ]:
df = pd.read_csv("data")
df1 = pd.read_csv("data")